# 인스타그램 해시태그 크롤링
## 필요한 라이브러리 import

In [1]:
#%pip install selenium
#%pip install webdriver_manager
#%pip install lxml
#%pip install openpyxl
#%pip install --upgrade pandas
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from urllib.parse import quote_plus
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from lxml import html
import urllib.request
import requests
import time
import re
import random

## 해시태그 url 변수 저장

In [2]:
# 해시태그 검색하여 url 변수에 저장
def Search_hashtag(word):
    url = "https://www.instagram.com/explore/tags/" + str(word)
    return url

## 열린 페이지에서 첫 번째 게시물 클릭

In [3]:
def select_first(driver):
  try:
    first_post = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href^="/p/"]'))
    )
    first_post_click()
  except Exception as e:
    print("첫 번째 게시물 선택 오류: {e}")
    
  #first = driver.find_element(By.CSS_SELECTOR, "div._aagw")
  #time.sleep(3)

## 첫 게시물 클릭 후 다음 게시물 클릭

In [3]:
def move_next_post(driver):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "div._aaqg._aaqh"))
        )
        next_button.click()
    except Exception as e:
        print(f"다음 게시물로 이동 오류: {e}")
    # right = driver.find_element(By.CSS_SELECTOR, "div._aaqg._aaqh")
    # time.sleep(3)

## 크롤링 함수

In [5]:
# 함수 정의: 본문 내용, 작성일자, 좋아요 수, 해시태그 가져오기
def collect_links(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = soup.select('a[href^="/p/"]')
    return {'https://www.instagram.com' + link['href'] for link in links}

def get_content(driver, index):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 이미지 저장
    try:
        img_element = driver.find_element(By.CSS_SELECTOR, 'img[srcset]')
        img_src = img_element.get_attribute('src')
        urllib.request.urlretrieve(img_src, f'{index}.jpg')
    except Exception as e:
        print(f"이미지 저장 오류: {e}")
        img_src = None
    
    # 링크
    link = driver.current_url
    
    # 좋아요 수
    try:
        likes = soup.select_one('section.EDfFK.ygqzn span').text
    except:
        likes = None
    
    data = {
        "img_src":img_src,
        "link":link,
        "likes":likes
    }
    return data

## 로그인 함수

In [6]:
def login_instagram(driver, username, password):
    try:
        username_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='username']"))
        )
        username_input.send_keys(username)

        password_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='password']"))
        )
        password_input.send_keys(password)

        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))
        )
        login_button.click()
        return True
    except Exception as e:
        print(f"로그인 오류: {e}")
        return False

## 크롤링 시작

In [7]:
# 크롬 브라우저 열기
service = Service(ChromeDriverManager().install()) # 크롬드라이버 최신 버전 자동 설치
options = webdriver.ChromeOptions() # 크롬브라우저 옵션 설정
driver = webdriver.Chrome(service=service, options=options) # 크롬브라우저 제어
driver.implicitly_wait(10) # 페이지 로드까지 기다리는 시간, 초과하면 NoSuchElementException
driver.get('https://www.instagram.com')
time.sleep(3)

#인스타그램 로그인을 위한 계정 정보
username = ''
password = ''
if not login_instagram(driver, username, password):
    driver.quit()
    raise Exception("로그인 실패")

time.sleep(5)

# 해시태그 검색
word = input("검색할 해시태그를 입력하세요: ")
url = Search_hashtag(word)

# 검색 결과 페이지 열기
print(f"해시태그 {word}에 대한 검색 결과 페이지 열기")
driver.get(url)
time.sleep(10)

# 첫 번째 게시물 클릭
select_first(driver)

# 데이터 수집 시작
results = []
# 수집할 게시물의 수
target = 27
for i in range(target):
    start_post_time = time.time()
    print(f"{i+1}/{target} 번째 게시물 수집 중...")
    try:
        data = get_content(driver, i)
        results.append(data)
        move_next_post(driver)
    except Exception as e:
        print(f"게시물 수집 오류 : {e}")
        move_next_post(driver)
    time.sleep(5)
    
print(results[:2])

# 결과를 데이터프레임으로 저장
date = datetime.today().strftime('%Y-%m-%d')
results_df = pd.DataFrame(results)

# 엑셀 파일로 저장
results_df.to_excel('instagram_crawling.xlsx')

# WebDriver 종료
driver.quit()

해시태그 마뗑킴코디에 대한 검색 결과 페이지 열기
첫 번째 게시물 선택 오류: {e}
1/27 번째 게시물 수집 중...
이미지 저장 오류: 'NoneType' object has no attribute 'get_attribute'
게시물 수집 오류 : Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x0080B8E3+45827]
	(No symbol) [0x0079DCC4]
	(No symbol) [0x0069150F]
	(No symbol) [0x0066E133]
	(No symbol) [0x006F949F]
	(No symbol) [0x0070B8E6]
	(No symbol) [0x006F2B96]
	(No symbol) [0x006C6998]
	(No symbol) [0x006C751D]
	GetHandleVerifier [0x00AC4513+2899763]
	GetHandleVerifier [0x00B1793D+3240797]
	GetHandleVerifier [0x008913B4+593364]
	GetHandleVerifier [0x008982DC+621820]
	(No symbol) [0x007A70A4]
	(No symbol) [0x007A37A8]
	(No symbol) [0x007A3947]
	(No symbol) [0x007959FE]
	BaseThreadInitThunk [0x75AE7BA9+25]
	RtlInitializeExceptionChain [0x77ABBE3B+107]
	RtlClearBits [0x77ABBDBF+191]

다음 게시물로 이동 오류: Message: no such window: target window already closed
from unknown error: w

KeyboardInterrupt: 

In [ ]:
# 변환할 엑셀 파일 경로
excel_file = r'\#merged_Martinkim.xlsx'

# CSV 파일로 저장할 경로
csv_file = r'\마뗑킴\#merged_Martinkim.csv'

# 엑셀 파일 읽기
df = pd.read_excel(excel_file)

# CSV 파일로 저장
df.to_csv(csv_file, index=False)

In [1]:
df = pd.read_excel(r'\아디다스\#merged_adidas.xlsx')
df

NameError: name 'pd' is not defined